<a href="https://colab.research.google.com/github/datjandra/GPDoc/blob/main/gpdoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTRUCTIONS:

1. Get a HuggingFace token from huggingface.co. Go to Secrets menu on left sidebar, the one with the key icon. Add a new secret named HF_TOKEN and paste the token value. Grant notebook access to secrets. You only have to do this one time and this token can be used in other notebooks.
2. In left navbar click Files icon. right click anywhere in the Files section and create a new folder called patient_data.
3. Add doctor's notes in .txt file format to patient_data folder. Below is an example note. **Don't** use notes with real patient data!  
4. Run the code below and enter questions. Sample questions are given below. To end the session, enter "bye", "quit" or "exit".
5. If you find issues like missing imports then try restarting session under Runtime menu.


SAMPLE QUESTIONS:
1. Can you provide more details about the patient's medical history, including past illnesses or surgeries?
2. What are the current dosages and frequencies of the medications prescribed?
3. Are there any new symptoms or changes in the patient's health status that we should be aware of?


SAMPLE NOTE:
```
Personal Data and Overall Health
September 1, 2006

Problems
Asthma, unspecified without mention of status asthmaticus

Medications
Flovent 110 mcg/inh aerosol with adapter Glaxo Wellcome
Singulair 5mg tablet chewable Merck & Co. Inc
Albuterol Sulfate 4mg tablet ESI Lederle Generics
Sulfamethoxazole-Trimethoprim 80mg-16 mg/ml solution ESI Lederle Generics
Penicillin V Potassium V potassium 500mg tablet UDL Laboratories Inc

Allergies
Lactose
Shellfish


Reason for Visit
Shortness of breath following pneumonia.

Narrative History
Patient diagnosed with Asthma 01/03. Has severe shortness of breath, fainting spells 2 week after pneumonia diagnosis.

Review of Systems
Shortness of breath. Coughing. Swimming in the head. Chest pain.

Vital Signs
BLOOD PRESSURE   119/61
PULSE   125.7
TEMPERATURE   97.8 F
WEIGHT   114.4 lb
O2 SAT   81


Exam
Pharynx normal.? PERRL, EOM full.
Neck without nodes.
Chest is clear to P&A.
Cor: normal S1, S2, without gallop or murmur.


External Labs
CXR reviewed: clear.

Impression
Generally doing well.? Low 02 sat. Pain reasonably well controlled.? No signs of active infection.

Plan
1.)? Nebulizer
2.)? RTC 2 weeks.
```



In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow

In [4]:
import os
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext, load_index_from_storage, set_global_service_context
from llama_index.llms import HuggingFaceInferenceAPI
from google.colab import userdata

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
  llm = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=userdata.get('HF_TOKEN'))
  service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm, embed_model="local")
  set_global_service_context(service_context)
  storage_context = StorageContext.from_defaults()
  docs = SimpleDirectoryReader("/content/patient_data").load_data()
  index = VectorStoreIndex.from_documents(docs, service_context=service_context, storage_context=storage_context)
  index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
  storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
  index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()
gpdoc = "GPDoc:"
exit_conditions = (":q", "quit", "exit", "bye")
print("Please start entering your questions below. To quit, enter one of these keywords: ", exit_conditions)
print()

while True:
  query = input("> ")
  if query in exit_conditions:
    print(gpdoc, "Good bye")
    break
  else:
    answer = query_engine.query(query)
    if answer:
      print(gpdoc, answer.response.strip())
      print()
    else:
      print(gpdoc, "Sorry I can't help you with that. Try rephrasing your question.")
      print()

Please start entering your questions below. To quit, enter one of these keywords:  (':q', 'quit', 'exit', 'bye')

> Can you tell me about the patent's symptoms?
GPDoc: The patient has been experiencing nausea, vomiting, and diarrhea for one day. She has been unable to keep up with her fluid losses due to these symptoms. At an outside institution, she was found to be very dehydrated and received IV fluids. She has been admitted to the ER in stable condition for further management of her dehydration.

> tell me more about the nausea
GPDoc: According to the patient's history of present illness, she has been experiencing nausea, vomiting, and diarrhea for one day. The nausea is not described in detail, but it is mentioned that it has been present along with vomiting and diarrhea. The patient's nausea may be related to the gastroenteritis that she has been experiencing.

> How about the vomiting and diarrhea?
GPDoc: The patient has been experiencing nausea, vomiting, and diarrhea for one da